<a href="https://colab.research.google.com/github/prasanna14200/Frontenddevelopmentprojects/blob/main/Copy_of_Week_3_Day_5_Meeting_Minutes_product.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create meeting minutes from an Audio file

I downloaded some Denver City Council meeting minutes and selected a portion of the meeting for us to transcribe. You can download it here:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing

If you'd rather work with the original data, the HuggingFace dataset is [here](https://huggingface.co/datasets/huuuyeah/meetingbank) and the audio can be downloaded [here](https://huggingface.co/datasets/huuuyeah/MeetingBank_Audio/tree/main).

The goal of this product is to use the Audio to generate meeting minutes, including actions.

For this project, you can either use the Denver meeting minutes, or you can record something of your own!


## Again - please note: 2 important pro-tips for using Colab:

**Pro-tip 1:**

The top of every colab has some pip installs. You may receive errors from pip when you run this, such as:

> gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.

These pip compatibility errors can be safely ignored; and while it's tempting to try to fix them by changing version numbers, that will actually introduce real problems!

**Pro-tip 2:**

In the middle of running a Colab, you might get an error like this:

> Runtime error: CUDA is required but not available for bitsandbytes. Please consider installing [...]

This is a super-misleading error message! Please don't try changing versions of packages...

This actually happens because Google has switched out your Colab runtime, perhaps because Google Colab was too busy. The solution is:

1. Kernel menu >> Disconnect and delete runtime
2. Reload the colab from fresh and Edit menu >> Clear All Outputs
3. Connect to a new T4 using the button at the top right
4. Select "View resources" from the menu on the top right to confirm you have a GPU
5. Rerun the cells in the colab, from the top down, starting with the pip installs

And all should work great - otherwise, ask me!

In [1]:
!pip install -q requests torch bitsandbytes transformers sentencepiece accelerate openai httpx==0.27.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 10.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curr

In [2]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google.colab import drive
from huggingface_hub import login
from google.colab import userdata
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
import torch

In [3]:
# Constants

AUDIO_MODEL = "whisper-1"
LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [4]:
# New capability - connect this Colab to my Google Drive
# See immediately below this for instructions to obtain denver_extract.mp3

drive.mount("/content/drive")
audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"

Mounted at /content/drive


# Download denver_extract.mp3

You can either use the same file as me, the extract from Denver city council minutes, or you can try your own..

If you want to use the same as me, then please download my extract here, and put this on your Google Drive:  
https://drive.google.com/file/d/1N_kpSojRR5RYzupz6nqM8hMSoEF_R7pU/view?usp=sharing


In [5]:
# Sign in to HuggingFace Hub

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

In [ ]:
# Sign in to OpenAI using Secrets in Colab

openai_api_key = userdata.get('OPENAI_API_KEY')
openai = OpenAI(api_key=openai_api_key)

In [6]:
pip install -U openai-whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 16.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803404 sha256=4e5dc1a0bea3ae3fbd4b778fdb911f1e77de85f9d3742417f63ed37dc63f1f20
  Stored in directory: /root/.cache/pip/wheels/2f/f2/ce/6eb23db4091d026238ce76703bd66da60b969d70bcc81d5d3a
Successfully built openai-whisper


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:

import whisper

# Path to the audio file in Google Drive
audio_filename = "/content/drive/MyDrive/llms/denver_extract.mp3"
model = whisper.load_model("base")  # or "medium", "large"
result = model.transcribe(audio_filename)
print(result["text"])

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 46.1MiB/s]


 kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason the back of the logo is considered water. So let you see the creation of the logo here. And yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of indigenous peoples day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th, please rise with the pledge of allegiance by Councilman Lopez. One makes me under God in the visible liberty 

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
# If you'd prefer to use an Open Source model, class student Youssef has contributed an open source version
# which I've added to the bottom of this colab

audio_file = open(audio_filename, "rb")
transcription = openai.audio.transcriptions.create(model=AUDIO_MODEL, file=audio_file, response_format="text")
print(transcription)

In [9]:
system_message = "You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown."
user_prompt = f"Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.\n{result}"

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]


In [10]:
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)

In [11]:
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="auto", quantization_config=quant_config)
outputs = model.generate(inputs, max_new_tokens=2000, streamer=streamer)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
{'text': " kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason the back of the logo is considered water. So let you see the creation of the logo here. And yes, so that basically kind of sums up the reason behind the logo and all t

In [13]:
response = tokenizer.decode(outputs[0])

In [14]:
display(Markdown(response))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are an assistant that produces minutes of meetings from transcripts, with summary, key discussion points, takeaways and action items with owners, in markdown.<|eot_id|><|start_header_id|>user<|end_header_id|>

Below is an extract transcript of a Denver council meeting. Please write minutes in markdown, including a summary with attendees, location and date; discussion points; takeaways; and action items with owners.
{'text': " kind of the confluence of this whole idea of the confluence week, the merging of two rivers, and as we've kind of seen recently in politics and in the world, there's a lot of situations where water is very important right now and it's a very big issue. So that is the reason the back of the logo is considered water. So let you see the creation of the logo here. And yes, so that basically kind of sums up the reason behind the logo and all the meanings behind the symbolism. And you'll hear a little bit more about our confluence week is basically highlighting all of these indigenous events and things that are happening around Denver so that we can kind of bring more people together and kind of share this whole idea of indigenous peoples day. So thank you. Thank you so much and thanks for your leadership. All right. Welcome to the Denver City Council meeting of Monday, October 9th, please rise with the pledge of allegiance by Councilman Lopez. One makes me under God in the visible liberty of the justice for all. All right. Thank you Councilman Lopez. Madam Secretary, roll call. Black, clerk, espinosa, here, Flynn, Gilmore, here, here, Cashman, here, can each, here, Lopez, new, here, or Teiga, here, Sussan, Mr. President, here, 11 present, 11 members present, we do have a quorum. Approval of the minutes, are there any corrections to the minutes of October 2nd? Seeing none, minutes of October 2nd, stand approved council announcement. So are there any announcements by members of council? Councilman Clark. Thank you, Mr. President. I just want to do invite everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen on Saturday, October 21st at 6 o'clock PM. It will move along Broadway from 3rd to Alameda. It's going to be a fun family friendly event. Everyone's invited to come down. Where costume? There will be candy for the kids and there are teaky zombies and 29 herses and all kinds of fun and funky stuff on the fun and funky part of Broadway. So please join us October 21st at 6 o'clock for the Broadway Halloween parade. Thank you, Mr. President. All right. Thank you, Councilman Clark. We'll be there. All right. Presentations. Madam Secretary, we have any presentations? None, Mr. President. Communications. Do we have any communications? None, Mr. President. We do have one proclamation this evening, proclamation 1127, an observance of the annual Indigenous People's Day in the City and County of Denver. Councilman Loplayes, would you please read it? Thank you, Mr. President. We're fried. Proclamation number 17, well let me just say this proclamation number 1127 series of 2017 and observance of the second annual Indigenous People's Day in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes that the Indigenous peoples have lived and flourished on the lands known as the Ametica since time immemorial and that Denver and the surrounding communities are built upon the ancestral homelands of numerous Indigenous tribes which include the southern Newt, the Yut Mountain, Yut tribes of Colorado and whereas the tribal homelands and seasonal encampments of the Arapoho and Shamb people along the banks of the Cherry Creek and South Plot River confluence gave bearing to the future settlements that would become the birthplace of the Mile High City. And whereas Colorado encompasses the ancestral homelands of 48 tribes in the City and County of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations. And whereas on October 3rd 2016 the City and County of Denver unanimously passed Council Bill 801 series of 2016 officially designating the second Monday of October of each year as Indigenous People's Day in Denver, Colorado. And whereas the Council of the City and County of Denver continues to recognize and value the vast contributions made to community, made to the community through Indigenous People's Knowledge, Science, Philosophy, Arts and Culture and through these contributions the City of Denver has developed and thrived. Whereas the Indigenous community especially Yut have made great efforts this year to draw attention to the contributions of Indigenous people including Confluence Week, drawing record of tennis to a National Indigenous Youth Leadership Conference leading conversations on inclusion with their peers and supporting increased Indigenous youth participation in science and engineering. Now therefore be it proclaimed by the Council of the City and County of Denver Section 1 that the Council of the City and County of Denver celebrates and honors the cultural and foundational contributions of Indigenous people to our history. Our past, present and future and continues to promote the education of the Denver community about these historic and contemporary contributions of Indigenous people. Section 2 at the City and County of Denver, Colorado does hereby observe October 9, 2017 as Indigenous People's Day. Section 3 at the clerk of the City and County of Denver shall attest and affix the seal of the City and County of Denver to this proclamation and that a copy be transmitted, excuse me, to the Denver American Indian Commission, the City and County of Denver School District number 1 and the Colorado Commission on Indian Affairs. Thank you Councilman Lopez, your motion to adopt. Mr. President, I move that proclamation number 1127, series of 2017 be adopted. All right, it has been moved and second, it comes with a council councilman Lopez. Thank you Mr. President. It gives me a lot of pleasure and pride to read this proclamation officially for the third time but as Indigenous people's day in Denver officially for the second time. It is, it's always awesome to be able to see not just this proclamation come through, come by my desk but to see so many different people from our community in our council chambers. It was a very beautiful piece of artwork that you presented to us earlier and it is exactly the spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous People's Day when we sat down and wrote it and as a community we couldn't think of anything else to begin except for the confluence of the two rivers and those confluence of the two rivers created such a great city and we live in such an amazing city and we we're all proud of it and sometimes we and a lot of people from all over the country are out of the world are proud of it and sometimes a little too proud of it is telling them to go back home. But I'm kidding when I say that but the really nice thing about this is that we are celebrating Indigenous People's Day out of pride for who we are, who we are as a city and the contributions of Indigenous people to the city not out of spite, not out of a replacement of one culture over the other or out of contempt or disrespect. I think of a quote that Sisso Chavez made very very popular and it stuck with me for a very long time and anytime I have the opportunity I speak in front of children and especially children in our community that they often second guests themselves and where they're coming from, who they are and I always say that it's very important to be proud of who you're from and the quote that I use Sisso Chavez is pride in one's own culture does not require contempt or disrespect of another and that's very important. It's very important for us to recognize that no matter who we are where we come from in this society that your pride in your own culture doesn't require the contempt or disrespect of another. Amen, what a year to be for that to just sit on our shoulders for a while for us to think about. And so I wanted to just to thank you all, I think the commission, there's going to be a couple individuals that are going to come speak thank you for your art, your lovely artwork for us to see what's in your heart and what now has become a probably going to be a very important symbol for the community and also just for the work, the daily work every single day we still have a lot of brothers and sisters whose ancestors once lived in these lands freely now stand on street corners right in poverty without access to services right without access to sobriety or even housing or jobs and what a what a what a cruel way to pay back a culture that has paved the way for the city to be built upon its shores right so we have a lot of work to do and these kind of proclamations in this day is not a day off it's a day on and then right and addressing those those those critical issues so I know that my colleagues are very supportive I'm going to ask you to support this proclamation as I know you always have done in the past I'm very proud of today oh and we made time magazine and newsweek once again today as being a leader in terms of the cities that are celebrating Indigenous peoples day I wanted to make a point out of that thank you uh councilman Lopez and thank you for sponsoring this council martega Mr. President I want to ask that my name be added I don't think I could um add much more to what councilman Lopez has shared with us I want to thank him for bringing this forward and really just appreciate all the contributions that our Native American community has contributed to this great city and great state I worked in the lieutenant governor's office when the commission on Indian affairs was created and had the benefit of being able to go down to the four corners for a peace treaty signing ceremony between the uts and the command cheese that had been sort of at odds with each other for about a hundred years and just being able to participate in that powwow was was pretty awesome so um and for those of you who continue to participate in the annual powwow it's it's such a great opportunity for everybody else to enjoy so many of the contributions of the culture I mean to see that the dance continues to be carried on as well as as the native language from generation to generation is just so incredible because in so many cultures you know people have come here and um assimilated to the you know the norms here and they lose their language and and lose a lot of the culture and in the native community that that hasn't happened that has that you know commitment to just passing that on from generation to generation is is so important and so I'm happy to be a co-sponsor of this tonight thank you all right thank you councilman or take a councilwoman can you thank you very much and I also want to thank my colleague for bringing this forward and I I just wanted to to say a word to the artist about how beautiful and moving I thought this logo was and your description of it um and I think one of the things um that is clear is you know the words sometimes don't convey the power of imagery or music or the other pieces that make up culture and so I think the art is so important and when you talked about water I was also thinking about land and I guess I just wanted to say thank you um many of the native American peoples of Colorado have been at the forefront are actually nationally of defending some of the the um public lands that have been protected over the last few years that are under attack right now and their places that you uh the communities have fought to protect but that everyone gets to enjoy and so I just think that it's an example of where cultural preservation intersects with environmental protection with you know recreation and all of the other ways that that public lands are so important and so I think um I just wanted to say thank you for that because I think we have some very sacred places in our country that are at risk right now and um so as we celebrate I appreciate that there's still a piece of resistance in here and I think that um I just want to mention a solidarity and I mentioned a feeling of solidarity with that resistance so thank you and uh happy confluence week thank you uh councilman can each uh and see you know other comments I'll just say a couple and um in a time of such divisive um ugliness and just despicable behavior from our leadership um the reason I'm so supportive of indigenous peoples days because it means inclusivity it means respecting all respecting those who have been silenced um uh on purpose for a long time and whose history has not been told and so we celebrate inclusivity in the face of such um such evil types honestly",'segments': [{'id': 0,'seek': 0,'start': 0.0, 'end': 5.04, 'text':'kind of the confluence of this whole idea of the confluence week, the merging of two rivers,', 'tokens': [50364, 733, 295, 264, 1497, 40432, 295, 341, 1379, 1558, 295, 264, 1497, 40432, 1243, 11, 264, 44559, 295, 732, 18361, 11, 50616], 'temperature': 0.0, 'avg_logprob': -0.23103076809055204, 'compression_ratio': 1.7572815533980584, 'no_speech_prob': 0.02853783220052719}, {'id': 1,'seek': 0,'start': 5.04, 'end': 13.68, 'text': " and as we've kind of seen recently in politics and in the world, there's a lot of situations", 'tokens': [50616, 293, 382, 321, 600, 733, 295, 1612, 3938, 294, 7341, 293, 294, 264, 1002, 11, 456, 311, 257, 688, 295, 6851, 51048], 'temperature': 0.0, 'avg_logprob': -0.23103076809055204, 'compression_ratio': 1.7572815533980584, 'no_speech_prob': 0.02853783220052719}, {'id': 2,'seek': 0,'start': 13.68, 'end': 18.12, 'text': " where water is very important right now and it's a very big issue. So that is the reason", 'tokens': [51048, 689, 1281, 307, 588, 1021, 558, 586, 293, 309, 311, 257, 588, 955, 2734, 13, 407, 300, 307, 264, 1778, 51270], 'temperature': 0.0, 'avg_logprob': -0.23103076809055204, 'compression_ratio': 1.7572815533980584, 'no_speech_prob': 0.02853783220052719}, {'id': 3,'seek': 0,'start': 18.12, 'end': 26.16, 'text':'the back of the logo is considered water. So let you see the creation of the logo here.', 'tokens': [51270, 264, 646, 295, 264, 9699, 307, 4888, 1281, 13, 407, 718, 291, 536, 264, 8016, 295, 264, 9699, 510, 13, 51672], 'temperature': 0.0, 'avg_logprob': -0.23103076809055204, 'compression_ratio': 1.7572815533980584, 'no_speech_prob': 0.02853783220052719}, {'id': 4,'seek': 3000,'start': 30.0, 'end': 46.24, 'text':'And yes, so that basically kind of sums up the reason behind the logo and all the meanings', 'tokens': [50364, 400, 2086, 11, 370, 300, 1936, 733, 295, 34499, 493, 264, 1778, 2261, 264, 9699, 293, 439, 264, 28138, 51176], 'temperature': 0.0, 'avg_logprob': -0.19173690020027806, 'compression_ratio': 1.5511363636363635, 'no_speech_prob': 0.006398750934749842}, {'id': 5,'seek': 3000,'start': 46.24, 'end': 53.16, 'text': " behind the symbolism. And you'll hear a little bit more about our confluence week is basically", 'tokens': [51176, 2261, 264, 47061, 13, 400, 291, 603, 1568, 257, 707, 857, 544, 466, 527, 1497, 40432, 1243, 307, 1936, 51522], 'temperature': 0.0, 'avg_logprob': -0.19173690020027806, 'compression_ratio': 1.5511363636363635, 'no_speech_prob': 0.006398750934749842}, {'id': 6,'seek': 3000,'start': 53.16, 'end': 57.04, 'text':'highlighting all of these indigenous events and things that are happening around Denver', 'tokens': [51522, 26551, 439, 295, 613, 15511, 3931, 293, 721, 300, 366, 2737, 926, 26270, 51716], 'temperature': 0.0, 'avg_logprob': -0.19173690020027806, 'compression_ratio': 1.5511363636363635, 'no_speech_prob': 0.006398750934749842}, {'id': 7,'seek': 5704,'start': 57.04, 'end': 62.4, 'text':'so that we can kind of bring more people together and kind of share this whole idea of', 'tokens': [50364, 370, 300, 321, 393, 733, 295, 1565, 544, 561, 1214, 293, 733, 295, 2073, 341, 1379, 1558, 295, 50632], 'temperature': 0.0, 'avg_logprob': -0.21285046757878484, 'compression_ratio': 1.4928909952606635, 'no_speech_prob': 0.052450623363256454}, {'id': 8,'seek': 5704,'start': 62.4, 'end': 66.8, 'text':'indigenous peoples day. So thank you. Thank you so much and thanks for your leadership.', 'tokens': [50632, 15511, 16915, 786, 13, 407, 1309, 291, 13, 1044, 291, 370, 709, 293, 3231, 337, 428, 5848, 13, 50852], 'temperature': 0.0, 'avg_logprob': -0.21285046757878484, 'compression_ratio': 1.4928909952606635, 'no_speech_prob': 0.052450623363256454}, {'id': 9,'seek': 5704,'start': 68.16, 'end': 73.52, 'text':'All right. Welcome to the Denver City Council meeting of Monday, October 9th,', 'tokens': [50920, 1057, 558, 13, 4027, 281, 264, 26270, 4392, 7076, 3440, 295, 8138, 11, 7617, 1722, 392, 11, 51188], 'temperature': 0.0, 'avg_logprob': -0.21285046757878484, 'compression_ratio': 1.4928909952606635, 'no_speech_prob': 0.052450623363256454}, {'id': 10,'seek': 5704,'start': 73.52, 'end': 76.64, 'text':'please rise with the pledge of allegiance by Councilman Lopez.', 'tokens': [51188, 1767, 6272, 365, 264, 26819, 295, 44706, 538, 7076, 1601, 36077, 13, 51344], 'temperature': 0.0, 'avg_logprob': -0.21285046757878484, 'compression_ratio': 1.4928909952606635, 'no_speech_prob': 0.052450623363256454}, {'id': 11,'seek': 8704,'start': 87.04, 'end': 92.16000000000001, 'text':'One makes me under God in the visible liberty of the justice for all.', 'tokens': [50364, 1485, 1669, 385, 833, 1265, 294, 264, 8974, 22849, 295, 264, 6118, 337, 439, 13, 50620], 'temperature': 0.0, 'avg_logprob': -0.5952443209561434, 'compression_ratio': 1.2702702702702702, 'no_speech_prob': 0.15752170979976654}, {'id': 12,'seek': 8704,'start': 96.72, 'end': 99.28, 'text':'All right. Thank you Councilman Lopez. Madam Secretary, roll call.', 'tokens': [50848, 1057, 558, 13, 1044, 291, 7076, 1601, 36077, 13, 18490, 9126, 11, 3373, 818, 13, 50976], 'temperature': 0.0, 'avg_logprob': -0.5952443209561434, 'compression_ratio': 1.2702702702702702, 'no_speech_prob': 0.15752170979976654}, {'id': 13,'seek': 8704,'start': 100.64, 'end': 109.44000000000001, 'text':'Black, clerk, espinosa, here, Flynn, Gilmore, here,', 'tokens': [51044, 4076, 11, 31402, 11, 7089, 259, 6447, 11, 510, 11, 40391, 11, 17654, 3138, 11, 510, 11, 51484], 'temperature': 0.0, 'avg_logprob': -0.5952443209561434, 'compression_ratio': 1.2702702702702702, 'no_speech_prob': 0.15752170979976654}, {'id': 14,'seek': 10944,'start': 110.39999999999999, 'end': 117.36, 'text':'here, Cashman, here, can each, here, Lopez, new, here, or Teiga, here,', 'tokens': [50412, 510, 11, 27016, 1601, 11, 510, 11, 393, 1184, 11, 510, 11, 36077, 11, 777, 11, 510, 11, 420, 1989, 9900, 11, 510, 11, 50760], 'temperature': 0.0, 'avg_logprob': -0.3852577611019737, 'compression_ratio': 1.7295918367346939, 'no_speech_prob': 0.04879871383309364}, {'id': 15,'seek': 10944,'start': 117.36, 'end': 125.12, 'text':'Sussan, Mr. President, here, 11 present, 11 members present, we do have a quorum.', 'tokens': [50760, 318, 2023, 282, 11, 2221, 13, 3117, 11, 510, 11, 2975, 1974, 11, 2975, 2679, 1974, 11, 321, 360, 362, 257, 421, 36543, 13, 51148], 'temperature': 0.0, 'avg_logprob': -0.3852577611019737, 'compression_ratio': 1.7295918367346939, 'no_speech_prob': 0.04879871383309364}, {'id': 16,'seek': 10944,'start': 125.12, 'end': 129.68, 'text':'Approval of the minutes, are there any corrections to the minutes of October 2nd?', 'tokens': [51148, 29551, 3337, 295, 264, 2077, 11, 366, 456, 604, 36406, 281, 264, 2077, 295, 7617, 568, 273, 30, 51376], 'temperature': 0.0, 'avg_logprob': -0.3852577611019737, 'compression_ratio': 1.7295918367346939, 'no_speech_prob': 0.04879871383309364}, {'id': 17,'seek': 10944,'start': 130.72, 'end': 135.6, 'text':'Seeing none, minutes of October 2nd, stand approved council announcement. So are there any announcements', 'tokens': [51428, 19703, 6022, 11, 2077, 295, 7617, 568, 273, 11, 1463, 10826, 9209, 12847, 13, 407, 366, 456, 604, 23785, 51672], 'temperature': 0.0, 'avg_logprob': -0.3852577611019737, 'compression_ratio': 1.7295918367346939, 'no_speech_prob': 0.04879871383309364}, {'id': 18,'seek': 13560,'start': 135.6, 'end': 145.35999999999999, 'text':'by members of council? Councilman Clark. Thank you, Mr. President. I just want to do invite', 'tokens': [50364, 538, 2679, 295, 9209, 30, 7076, 1601, 18572, 13, 1044, 291, 11, 2221, 13, 3117, 13, 286, 445, 528, 281, 360, 7980, 50852], 'temperature': 0.0, 'avg_logprob': -0.16105720677326635, 'compression_ratio': 1.4609375, 'no_speech_prob': 0.02330799028277397}, {'id': 19,'seek': 13560,'start': 145.35999999999999, 'end': 151.92, 'text':'everyone down to the first ever Halloween parade on Broadway in Lucky District 7. It will happen', 'tokens': [50852, 1518, 760, 281, 264, 700, 1562, 13860, 26128, 322, 19414, 294, 26639, 14374, 1614, 13, 467, 486, 1051, 51180], 'temperature': 0.0, 'avg_logprob': -0.16105720677326635, 'compression_ratio': 1.4609375, 'no_speech_prob': 0.02330799028277397}, {'id': 20,'seek': 13560,'start': 151.92, 'end': 159.12, 'text': " on Saturday, October 21st at 6 o'clock PM. It will move along Broadway from 3rd to Alameda. It's", 'tokens': [51180, 322, 8803, 11, 7617, 5080, 372, 412, 1386, 277, 6, 9023, 12499, 13, 467, 486, 1286, 2051, 19414, 490, 805, 7800, 281, 967, 3475, 64, 13, 467, 311, 51540], 'temperature': 0.0, 'avg_logprob': -0.16105720677326635, 'compression_ratio': 1.4609375, 'no_speech_prob': 0.02330799028277397}, {'id': 21,'seek': 13560,'start': 159.12, 'end': 163.84, 'text': " going to be a fun family friendly event. Everyone's invited to come down. Where costume?", 'tokens': [51540, 516, 281, 312, 257, 1019, 1605, 9208, 2280, 13, 5198, 311, 9185, 281, 808, 760, 13, 2305, 14850, 30, 51776], 'temperature': 0.0, 'avg_logprob': -0.16105720677326635, 'compression_ratio': 1.4609375, 'no_speech_prob': 0.02330799028277397}, {'id': 22,'seek': 16384,'start': 164.8, 'end': 174.24, 'text':'There will be candy for the kids and there are teaky zombies and 29 herses and all kinds of fun', 'tokens': [50412, 821, 486, 312, 11237, 337, 264, 2301, 293, 456, 366, 535, 15681, 24230, 293, 9413, 720, 6196, 293, 439, 3685, 295, 1019, 50884], 'temperature': 0.0, 'avg_logprob': -0.1594031810760498, 'compression_ratio': 1.5925925925925926, 'no_speech_prob': 0.06633295118808746}, {'id': 23,'seek': 16384,'start': 174.24, 'end': 180.88, 'text': " and funky stuff on the fun and funky part of Broadway. So please join us October 21st at 6 o'clock", 'tokens': [50884, 293, 33499, 1507, 322, 264, 1019, 293, 33499, 644, 295, 19414, 13, 407, 1767, 3917, 505, 7617, 5080, 372, 412, 1386, 277, 6, 9023, 51216], 'temperature': 0.0, 'avg_logprob': -0.1594031810760498, 'compression_ratio': 1.5925925925925926, 'no_speech_prob': 0.06633295118808746}, {'id': 24,'seek': 16384,'start': 180.88, 'end': 185.28, 'text':'for the Broadway Halloween parade. Thank you, Mr. President. All right. Thank you, Councilman Clark.', 'tokens': [51216, 337, 264, 19414, 13860, 26128, 13, 1044, 291, 11, 2221, 13, 3117, 13, 1057, 558, 13, 1044, 291, 11, 7076, 1601, 18572, 13, 51436], 'temperature': 0.0, 'avg_logprob': -0.1594031810760498, 'compression_ratio': 1.5925925925925926, 'no_speech_prob': 0.06633295118808746}, {'id': 25,'seek': 16384,'start': 186.08, 'end': 192.24, 'text': " We'll be there. All right. Presentations. Madam Secretary, we have any presentations? None,", 'tokens': [51476, 492, 603, 312, 456, 13, 1057, 558, 13, 33253, 763, 13, 18490, 9126, 11, 321, 362, 604, 18964, 30, 14492, 11, 51784], 'temperature': 0.0, 'avg_logprob': -0.1594031810760498, 'compression_ratio': 1.5925925925925926, 'no_speech_prob': 0.06633295118808746}, {'id': 26,'seek': 19224,'start': 192.24, 'end': 197.12, 'text':'Mr. President. Communications. Do we have any communications? None, Mr. President. We do have one', 'tokens': [50364, 2221, 13, 3117, 13, 38998, 13, 1144, 321, 362, 604, 15163, 30, 14492, 11, 2221, 13, 3117, 13, 492, 360, 362, 472, 50608], 'temperature': 0.0, 'avg_logprob': -0.25018558096378407, 'compression_ratio': 1.6008771929824561, 'no_speech_prob': 0.017157716676592827}, {'id': 27,'seek': 19224,'start': 197.12, 'end': 204.96, 'text': " proclamation this evening, proclamation 1127, an observance of the annual Indigenous People's", 'tokens': [50608, 447, 41411, 341, 5634, 11, 447, 41411, 2975, 10076, 11, 364, 9951, 719, 295, 264, 9784, 22699, 3432, 311, 51000], 'temperature': 0.0, 'avg_logprob': -0.25018558096378407, 'compression_ratio': 1.6008771929824561, 'no_speech_prob': 0.017157716676592827}, {'id': 28,'seek': 19224,'start': 205.84, 'end': 209.44, 'text':'Day in the City and County of Denver. Councilman Loplayes, would you please read it?', 'tokens': [51044, 5226, 294, 264, 4392, 293, 6658, 295, 26270, 13, 7076, 1601, 6130, 2858, 279, 11, 576, 291, 1767, 1401, 309, 30, 51224], 'temperature': 0.0, 'avg_logprob': -0.25018558096378407, 'compression_ratio': 1.6008771929824561, 'no_speech_prob': 0.017157716676592827}, {'id': 29,'seek': 19224,'start': 210.4, 'end': 216.72, 'text': " Thank you, Mr. President. We're fried. Proclamation number 17, well let me just say this", 'tokens': [51272, 1044, 291, 11, 2221, 13, 3117, 13, 492, 434, 10425, 13, 1705, 41411, 1230, 3282, 11, 731, 718, 385, 445, 584, 341, 51588], 'temperature': 0.0, 'avg_logprob': -0.25018558096378407, 'compression_ratio': 1.6008771929824561, 'no_speech_prob': 0.017157716676592827}, {'id': 30,'seek': 21672,'start': 217.44, 'end': 224.24, 'text': " proclamation number 1127 series of 2017 and observance of the second annual Indigenous People's Day", 'tokens': [50400, 447, 41411, 1230, 2975, 10076, 2638, 295, 6591, 293, 9951, 719, 295, 264, 1150, 9784, 22699, 3432, 311, 5226, 50740], 'temperature': 0.0, 'avg_logprob': -0.20421968973599947, 'compression_ratio': 1.690391459074733, 'no_speech_prob': 0.03300820663571358}, {'id': 31,'seek': 21672,'start': 224.24, 'end': 228.72, 'text':'in the City and County of Denver. Whereas the Council of the City and County of Denver recognizes', 'tokens': [50740, 294, 264, 4392, 293, 6658, 295, 26270, 13, 13813, 264, 7076, 295, 264, 4392, 293, 6658, 295, 26270, 26564, 50964], 'temperature': 0.0, 'avg_logprob': -0.20421968973599947, 'compression_ratio': 1.690391459074733, 'no_speech_prob': 0.03300820663571358}, {'id': 32,'seek': 21672,'start': 228.72, 'end': 233.44, 'text':'that the Indigenous peoples have lived and flourished on the lands known as the Ametica since time', 'tokens': [50964, 300, 264, 22699, 16915, 362, 5152, 293, 7693, 4729, 322, 264, 5949, 2570, 382, 264, 2012, 302, 2262, 1670, 565, 51200], 'temperature': 0.0, 'avg_logprob': -0.20421968973599947, 'compression_ratio': 1.690391459074733, 'no_speech_prob': 0.03300820663571358}, {'id': 33,'seek': 21672,'start': 234.07999999999998, 'end': 240.48, 'text':'immemorial and that Denver and the surrounding communities are built upon the ancestral homelands', 'tokens': [51232, 3397, 443, 5181, 293, 300, 26270, 293, 264, 11498, 4456, 366, 3094, 3564, 264, 40049, 3655, 338, 2967, 51552], 'temperature': 0.0, 'avg_logprob': -0.20421968973599947, 'compression_ratio': 1.690391459074733, 'no_speech_prob': 0.03300820663571358}, {'id': 34,'seek': 21672,'start': 240.48, 'end': 245.44, 'text':'of numerous Indigenous tribes which include the southern Newt, the Yut Mountain,', 'tokens': [51552, 295, 12546, 22699, 19035, 597, 4090, 264, 13456, 1873, 83, 11, 264, 398, 325, 15586, 11, 51800], 'temperature': 0.0, 'avg_logprob': -0.20421968973599947, 'compression_ratio': 1.690391459074733, 'no_speech_prob': 0.03300820663571358}, {'id': 35,'seek': 24544,'start': 245.44, 'end': 250.48, 'text':'Yut tribes of Colorado and whereas the tribal homelands and seasonal encampments of the', 'tokens': [50364, 398, 325, 19035, 295, 15786, 293, 9735, 264, 20958, 3655, 338, 2967, 293, 27421, 2058, 1215, 1117, 295, 264, 50616], 'temperature': 0.0, 'avg_logprob': -0.20643967563666185, 'compression_ratio': 1.7065217391304348, 'no_speech_prob': 0.0042572482489049435}, {'id': 36,'seek': 24544,'start': 250.48, 'end': 256.24, 'text':'Arapoho and Shamb people along the banks of the Cherry Creek and South Plot River confluence', 'tokens': [50616, 18601, 2259, 1289, 293, 1160, 2173, 561, 2051, 264, 10237, 295, 264, 34831, 24200, 293, 4242, 2149, 310, 8640, 1497, 40432, 50904], 'temperature': 0.0, 'avg_logprob': -0.20643967563666185, 'compression_ratio': 1.7065217391304348, 'no_speech_prob': 0.0042572482489049435}, {'id': 37,'seek': 24544,'start': 256.24, 'end': 260.48, 'text':'gave bearing to the future settlements that would become the birthplace of the Mile High City.', 'tokens': [50904, 2729, 17350, 281, 264, 2027, 35558, 300, 576, 1813, 264, 3965, 6742, 295, 264, 47651, 5229, 4392, 13, 51116], 'temperature': 0.0, 'avg_logprob': -0.20643967563666185, 'compression_ratio': 1.7065217391304348, 'no_speech_prob': 0.0042572482489049435}, {'id': 38,'seek': 24544,'start': 262.24, 'end': 268.15999999999997, 'text':'And whereas Colorado encompasses the ancestral homelands of 48 tribes in the City and County', 'tokens': [51204, 400, 9735, 15786, 49866, 264, 40049, 3655, 338, 2967, 295, 11174, 19035, 294, 264, 4392, 293, 6658, 51500], 'temperature': 0.0, 'avg_logprob': -0.20643967563666185, 'compression_ratio': 1.7065217391304348, 'no_speech_prob': 0.0042572482489049435}, {'id': 39,'seek': 24544,'start': 268.15999999999997, 'end': 273.6, 'text':'of Denver and surrounding communities are home to the descendants of approximately 100 tribal nations.', 'tokens': [51500, 295, 26270, 293, 11498, 4456, 366, 1280, 281, 264, 31693, 295, 10447, 2319, 20958, 11035, 13, 51772], 'temperature': 0.0, 'avg_logprob': -0.20643967563666185, 'compression_ratio': 1.7065217391304348, 'no_speech_prob': 0.0042572482489049435}, {'id': 40,'seek': 27360,'start': 274.56, 'end': 282.0, 'text':'And whereas on October 3rd 2016 the City and County of Denver unanimously passed', 'tokens': [50412, 400, 9735, 322, 7617, 805, 7800, 6549, 264, 4392, 293, 6658, 295, 26270, 48733, 4678, 50784], 'temperature': 0.0, 'avg_logprob': -0.1267118579462955, 'compression_ratio': 1.6081081081081081, 'no_speech_prob': 0.0054638381116092205}, {'id': 41,'seek': 27360,'start': 282.0, 'end': 288.56, 'text':'Council Bill 801 series of 2016 officially designating the second Monday of October of each', 'tokens': [50784, 7076, 5477, 4688, 16, 2638, 295, 6549, 12053, 1715, 990, 264, 1150, 8138, 295, 7617, 295, 1184, 51112], 'temperature': 0.0, 'avg_logprob': -0.1267118579462955, 'compression_ratio': 1.6081081081081081, 'no_speech_prob': 0.0054638381116092205}, {'id': 42,'seek': 27360,'start': 288.56, 'end': 295.84000000000003, 'text': " year as Indigenous People's Day in Denver, Colorado. And whereas the Council of the City and", 'tokens': [51112, 1064, 382, 22699, 3432, 311, 5226, 294, 26270, 11, 15786, 13, 400, 9735, 264, 7076, 295, 264, 4392, 293, 51476], 'temperature': 0.0, 'avg_logprob': -0.1267118579462955, 'compression_ratio': 1.6081081081081081, 'no_speech_prob': 0.0054638381116092205}, {'id': 43,'seek': 27360,'start': 295.84000000000003, 'end': 303.20000000000005, 'text':'County of Denver continues to recognize and value the vast contributions made to community,', 'tokens': [51476, 6658, 295, 26270, 6515, 281, 5521, 293, 2158, 264, 8369, 15725, 1027, 281, 1768, 11, 51844], 'temperature': 0.0, 'avg_logprob': -0.1267118579462955, 'compression_ratio': 1.6081081081081081, 'no_speech_prob': 0.0054638381116092205}, {'id': 44,'seek': 30320,'start': 303.2, 'end': 309.84, 'text': " made to the community through Indigenous People's Knowledge, Science, Philosophy, Arts and", 'tokens': [50364, 1027, 281, 264, 1768, 807, 22699, 3432, 311, 32906, 11, 8976, 11, 43655, 11, 12407, 293, 50696], 'temperature': 0.0, 'avg_logprob': -0.20196448432074654, 'compression_ratio': 1.6719367588932805, 'no_speech_prob': 0.0011804996756836772}, {'id': 45,'seek': 30320,'start': 309.84, 'end': 314.64, 'text':'Culture and through these contributions the City of Denver has developed and thrived.', 'tokens': [50696, 27539, 293, 807, 613, 15725, 264, 4392, 295, 26270, 575, 4743, 293, 23949, 937, 13, 50936], 'temperature': 0.0, 'avg_logprob': -0.20196448432074654, 'compression_ratio': 1.6719367588932805, 'no_speech_prob': 0.0011804996756836772}, {'id': 46,'seek': 30320,'start': 316.24, 'end': 322.64, 'text':'Whereas the Indigenous community especially Yut have made great efforts this year to draw', 'tokens': [51016, 13813, 264, 22699, 1768, 2318, 398, 325, 362, 1027, 869, 6484, 341, 1064, 281, 2642, 51336], 'temperature': 0.0, 'avg_logprob': -0.20196448432074654, 'compression_ratio': 1.6719367588932805, 'no_speech_prob': 0.0011804996756836772}, {'id': 47,'seek': 30320,'start': 322.64, 'end': 326.56, 'text':'attention to the contributions of Indigenous people including Confluence Week,', 'tokens': [51336, 3202, 281, 264, 15725, 295, 22699, 561, 3009, 11701, 40432, 12615, 11, 51532], 'temperature': 0.0, 'avg_logprob': -0.20196448432074654, 'compression_ratio': 1.6719367588932805, 'no_speech_prob': 0.0011804996756836772}, {'id': 48,'seek': 30320,'start': 327.12, 'end': 331.03999999999996, 'text':'drawing record of tennis to a National Indigenous Youth Leadership Conference', 'tokens': [51560, 6316, 2136, 295, 18118, 281, 257, 4862, 22699, 24312, 30577, 22131, 51756], 'temperature': 0.0, 'avg_logprob': -0.20196448432074654, 'compression_ratio': 1.6719367588932805, 'no_speech_prob': 0.0011804996756836772}, {'id': 49,'seek': 33104,'start': 331.04, 'end': 335.52000000000004, 'text':'leading conversations on inclusion with their peers and supporting', 'tokens': [50364, 5775, 7315, 322, 15874, 365, 641, 16739, 293, 7231, 50588], 'temperature': 0.0, 'avg_logprob': -0.1499015612479968, 'compression_ratio': 1.7652173913043478, 'no_speech_prob': 0.012790793552994728}, {'id': 50,'seek': 33104,'start': 335.52000000000004, 'end': 339.44, 'text':'increased Indigenous youth participation in science and engineering.', 'tokens': [50588, 6505, 22699, 7503, 13487, 294, 3497, 293, 7043, 13, 50784], 'temperature': 0.0, 'avg_logprob': -0.1499015612479968, 'compression_ratio': 1.7652173913043478, 'no_speech_prob': 0.012790793552994728}, {'id': 51,'seek': 33104,'start': 340.32, 'end': 345.44, 'text':'Now therefore be it proclaimed by the Council of the City and County of Denver Section 1', 'tokens': [50828, 823, 4412, 312, 309, 49091, 538, 264, 7076, 295, 264, 4392, 293, 6658, 295, 26270, 21804, 502, 51084], 'temperature': 0.0, 'avg_logprob': -0.1499015612479968, 'compression_ratio': 1.7652173913043478, 'no_speech_prob': 0.012790793552994728}, {'id': 52,'seek': 33104,'start': 346.0, 'end': 350.0, 'text':'that the Council of the City and County of Denver celebrates and honors the cultural and', 'tokens': [51112, 300, 264, 7076, 295, 264, 4392, 293, 6658, 295, 26270, 47182, 293, 26884, 264, 6988, 293, 51312], 'temperature': 0.0, 'avg_logprob': -0.1499015612479968, 'compression_ratio': 1.7652173913043478, 'no_speech_prob': 0.012790793552994728}, {'id': 53,'seek': 33104,'start': 350.0, 'end': 356.16, 'text':'foundational contributions of Indigenous people to our history. Our past, present and future', 'tokens': [51312, 32195, 15725, 295, 22699, 561, 281, 527, 2503, 13, 2621, 1791, 11, 1974, 293, 2027, 51620], 'temperature': 0.0, 'avg_logprob': -0.1499015612479968, 'compression_ratio': 1.7652173913043478, 'no_speech_prob': 0.012790793552994728}, {'id': 54,'seek': 35616,'start': 356.8, 'end': 362.0, 'text':'and continues to promote the education of the Denver community about these historic', 'tokens': [50396, 293, 6515, 281, 9773, 264, 3309, 295, 264, 26270, 1768, 466, 613, 13236, 50656], 'temperature': 0.0, 'avg_logprob': -0.17831976754324777, 'compression_ratio': 1.6180904522613064, 'no_speech_prob': 0.026722345501184464}, {'id': 55,'seek': 35616,'start': 362.64000000000004, 'end': 365.28000000000003, 'text':'and contemporary contributions of Indigenous people.', 'tokens': [50688, 293, 14878, 15725, 295, 22699, 561, 13, 50820], 'temperature': 0.0, 'avg_logprob': -0.17831976754324777, 'compression_ratio': 1.6180904522613064, 'no_speech_prob': 0.026722345501184464}, {'id': 56,'seek': 35616,'start': 367.28000000000003, 'end': 375.04, 'text':'Section 2 at the City and County of Denver, Colorado does hereby observe October 9, 2017', 'tokens': [50920, 21804, 568, 412, 264, 4392, 293, 6658, 295, 26270, 11, 15786, 775, 510, 2322, 11441, 7617, 1722, 11, 6591, 51308], 'temperature': 0.0, 'avg_logprob': -0.17831976754324777, 'compression_ratio': 1.6180904522613064, 'no_speech_prob': 0.026722345501184464}, {'id': 57,'seek': 35616,'start': 375.6, 'end': 382.0, 'text': " as Indigenous People's Day. Section 3 at the clerk of the City and County of Denver shall attest", 'tokens': [51336, 382, 22699, 3432, 311, 5226, 13, 21804, 805, 412, 264, 31402, 295, 264, 4392, 293, 6658, 295, 26270, 4393, 951, 377, 51656], 'temperature': 0.0, 'avg_logprob': -0.17831976754324777, 'compression_ratio': 1.6180904522613064, 'no_speech_prob': 0.026722345501184464}, {'id': 58,'seek': 38200,'start': 382.0, 'end': 386.64, 'text':'and affix the seal of the City and County of Denver to this proclamation and that a copy', 'tokens': [50364, 293, 2096, 970, 264, 12185, 295, 264, 4392, 293, 6658, 295, 26270, 281, 341, 447, 41411, 293, 300, 257, 5055, 50596], 'temperature': 0.0, 'avg_logprob': -0.2287971947782783, 'compression_ratio': 1.6553191489361703, 'no_speech_prob': 0.04950108006596565}, {'id': 59,'seek': 38200,'start': 386.64, 'end': 394.16, 'text':'be transmitted, excuse me, to the Denver American Indian Commission, the City and County of Denver', 'tokens': [50596, 312, 25355, 11, 8960, 385, 11, 281, 264, 26270, 2665, 6427, 10766, 11, 264, 4392, 293, 6658, 295, 26270, 50972], 'temperature': 0.0, 'avg_logprob': -0.2287971947782783, 'compression_ratio': 1.6553191489361703, 'no_speech_prob': 0.04950108006596565}, {'id': 60,'seek': 38200,'start': 394.16, 'end': 397.84, 'text':'School District number 1 and the Colorado Commission on Indian Affairs.', 'tokens': [50972, 5070, 14374, 1230, 502, 293, 264, 15786, 10766, 322, 6427, 21721, 13, 51156], 'temperature': 0.0, 'avg_logprob': -0.2287971947782783, 'compression_ratio': 1.6553191489361703, 'no_speech_prob': 0.04950108006596565}, {'id': 61,'seek': 38200,'start': 399.68, 'end': 401.92, 'text':'Thank you Councilman Lopez, your motion to adopt.', 'tokens': [51248, 1044, 291, 7076, 1601, 36077, 11, 428, 5394, 281, 6878, 13, 51360], 'temperature': 0.0, 'avg_logprob': -0.2287971947782783, 'compression_ratio': 1.6553191489361703, 'no_speech_prob': 0.04950108006596565}, {'id': 62,'seek': 38200,'start': 402.64, 'end': 407.92, 'text':'Mr. President, I move that proclamation number 1127, series of 2017 be adopted.', 'tokens': [51396, 2221, 13, 3117, 11, 286, 1286, 300, 447, 41411, 1230, 2975, 10076, 11, 2638, 295, 6591, 312, 12175, 13, 51660], 'temperature': 0.0, 'avg_logprob': -0.2287971947782783, 'compression_ratio': 1.6553191489361703, 'no_speech_prob': 0.04950108006596565}, {'id': 63,'seek': 40792,'start': 408.8, 'end': 412.8, 'text':'All right, it has been moved and second, it comes with a council councilman Lopez.', 'tokens': [50408, 1057, 558, 11, 309, 575, 668, 4259, 293, 1150, 11, 309, 1487, 365, 257, 9209, 9209, 1601, 36077, 13, 50608], 'temperature': 0.0, 'avg_logprob': -0.24547292969443582, 'compression_ratio': 1.6017699115044248, 'no_speech_prob': 0.04079349711537361}, {'id': 64,'seek': 40792,'start': 412.8, 'end': 418.96000000000004, 'text':'Thank you Mr. President. It gives me a lot of pleasure and pride to read this proclamation', 'tokens': [50608, 1044, 291, 2221, 13, 3117, 13, 467, 2709, 385, 257, 688, 295, 6834, 293, 10936, 281, 1401, 341, 447, 41411, 50916], 'temperature': 0.0, 'avg_logprob': -0.24547292969443582, 'compression_ratio': 1.6017699115044248, 'no_speech_prob': 0.04079349711537361}, {'id': 65,'seek': 40792,'start': 419.6, 'end': 425.68, 'text': " officially for the third time but as Indigenous people's day in Denver officially for the second", 'tokens': [50948, 12053, 337, 264, 2636, 565, 457, 382, 22699, 561, 311, 786, 294, 26270, 12053, 337, 264, 1150, 51252], 'temperature': 0.0, 'avg_logprob': -0.24547292969443582, 'compression_ratio': 1.6017699115044248, 'no_speech_prob': 0.04079349711537361}, {'id': 66,'seek': 40792,'start': 425.68, 'end': 434.56, 'text': " time. It is, it's always awesome to be able to see not just this proclamation come through,", 'tokens': [51252, 565, 13, 467, 307, 11, 309, 311, 1009, 3476, 281, 312, 1075, 281, 536, 406, 445, 341, 447, 41411, 808, 807, 11, 51696], 'temperature': 0.0, 'avg_logprob': -0.24547292969443582, 'compression_ratio': 1.6017699115044248, 'no_speech_prob': 0.04079349711537361}, {'id': 67,'seek': 43456,'start': 434.56, 'end': 441.36, 'text':'come by my desk but to see so many different people from our community in our council chambers.', 'tokens': [50364, 808, 538, 452, 10026, 457, 281, 536, 370, 867, 819, 561, 490, 527, 1768, 294, 527, 9209, 34513, 13, 50704], 'temperature': 0.0, 'avg_logprob': -0.11777715121998507, 'compression_ratio': 1.606694560669456, 'no_speech_prob': 0.008043183945119381}, {'id': 68,'seek': 43456,'start': 443.12, 'end': 448.64, 'text':'It was a very beautiful piece of artwork that you presented to us earlier and it is exactly the', 'tokens': [50792, 467, 390, 257, 588, 2238, 2522, 295, 15829, 300, 291, 8212, 281, 505, 3071, 293, 309, 307, 2293, 264, 51068], 'temperature': 0.0, 'avg_logprob': -0.11777715121998507, 'compression_ratio': 1.606694560669456, 'no_speech_prob': 0.008043183945119381}, {'id': 69,'seek': 43456,'start': 448.64, 'end': 455.12, 'text':'spirit that we drafted this proclamation and this actual, the ordinance that created Indigenous', 'tokens': [51068, 3797, 300, 321, 36288, 341, 447, 41411, 293, 341, 3539, 11, 264, 40260, 300, 2942, 22699, 51392], 'temperature': 0.0, 'avg_logprob': -0.11777715121998507, 'compression_ratio': 1.606694560669456, 'no_speech_prob': 0.008043183945119381}, {'id': 70,'seek': 43456,'start': 455.12, 'end': 460.88, 'text': " People's Day when we sat down and wrote it and as a community we couldn't think of anything else", 'tokens': [51392, 3432, 311, 5226, 562, 321, 3227, 760, 293, 4114, 309, 293, 382, 257, 1768, 321, 2809, 380, 519, 295, 1340, 1646, 51680], 'temperature': 0.0, 'avg_logprob': -0.11777715121998507, 'compression_ratio': 1.606694560669456, 'no_speech_prob': 0.008043183945119381}, {'id': 71,'seek': 46088,'start': 461.6, 'end': 466.32, 'text':'to begin except for the confluence of the two rivers and those confluence of the two rivers', 'tokens': [50400, 281, 1841, 3993, 337, 264, 1497, 40432, 295, 264, 732, 18361, 293, 729, 1497, 40432, 295, 264, 732, 18361, 50636], 'temperature': 0.0, 'avg_logprob': -0.15521717071533203, 'compression_ratio': 1.9344262295081966, 'no_speech_prob': 0.01092767808586359}, {'id': 72,'seek': 46088,'start': 466.32, 'end': 473.04, 'text': " created such a great city and we live in such an amazing city and we we're all proud of it and", 'tokens': [50636, 2942, 1270, 257, 869, 2307, 293, 321, 1621, 294, 1270, 364, 2243, 2307, 293, 321, 321, 434, 439, 4570, 295, 309, 293, 50972], 'temperature': 0.0, 'avg_logprob': -0.15521717071533203, 'compression_ratio': 1.9344262295081966, 'no_speech_prob': 0.01092767808586359}, {'id': 73,'seek': 46088,'start': 473.04, 'end': 478.32, 'text':'sometimes we and a lot of people from all over the country are out of the world are proud of it', 'tokens': [50972, 2171, 321, 293, 257, 688, 295, 561, 490, 439, 670, 264, 1941, 366, 484, 295, 264, 1002, 366, 4570, 295, 309, 51236], 'temperature': 0.0, 'avg_logprob': -0.15521717071533203, 'compression_ratio': 1.9344262295081966, 'no_speech_prob': 0.01092767808586359}, {'id': 74,'seek': 46088,'start': 478.32, 'end': 481.04, 'text':'and sometimes a little too proud of it is telling them to go back home.', 'tokens': [51236, 293, 2171, 257, 707, 886, 4570, 295, 309, 307, 3585, 552, 281, 352, 646, 1280, 13, 51372], 'temperature': 0.0, 'avg_logprob': -0.15521717071533203, 'compression_ratio': 1.9344262295081966, 'no_speech_prob': 0.01092767808586359}, {'id': 75,'seek': 48104,'start': 481.20000000000005, 'end': 492.16, 'text': " But I'm kidding when I say that but the really nice thing about this is that we are celebrating", 'tokens': [50372, 583, 286, 478, 9287, 562, 286, 584, 300, 457, 264, 534, 1481, 551, 466, 341, 307, 300, 321, 366, 15252, 50920], 'temperature': 0.0, 'avg_logprob': -0.16814046666242075, 'compression_ratio': 1.6033519553072626, 'no_speech_prob': 0.023974036797881126}, {'id': 76,'seek': 48104,'start': 492.16, 'end': 498.48, 'text': " Indigenous People's Day out of pride for who we are, who we are as a city and the contributions", 'tokens': [50920, 22699, 3432, 311, 5226, 484, 295, 10936, 337, 567, 321, 366, 11, 567, 321, 366, 382, 257, 2307, 293, 264, 15725, 51236], 'temperature': 0.0, 'avg_logprob': -0.16814046666242075, 'compression_ratio': 1.6033519553072626, 'no_speech_prob': 0.023974036797881126}, {'id': 77,'seek': 48104,'start': 498.48, 'end': 504.64000000000004, 'text':'of Indigenous people to the city not out of spite, not out of a replacement of one culture over', 'tokens': [51236, 295, 22699, 561, 281, 264, 2307, 406, 484, 295, 22794, 11, 406, 484, 295, 257, 14419, 295, 472, 3713, 670, 51544], 'temperature': 0.0, 'avg_logprob': -0.16814046666242075, 'compression_ratio': 1.6033519553072626, 'no_speech_prob': 0.023974036797881126}, {'id': 78,'seek': 50464,'start': 504.64, 'end': 515.92, 'text':'the other or out of contempt or disrespect. I think of a quote that Sisso Chavez made very', 'tokens': [50364, 264, 661, 420, 484, 295, 47202, 420, 27058, 13, 286, 519, 295, 257, 6513, 300, 318, 891, 78, 761, 25975, 1027, 588, 50928], 'temperature': 0.0, 'avg_logprob': -0.268391489982605, 'compression_ratio': 1.5337078651685394, 'no_speech_prob': 0.003574612084776163}, {'id': 79,'seek': 50464,'start': 515.92, 'end': 523.6, 'text':'very popular and it stuck with me for a very long time and anytime I have the opportunity', 'tokens': [50928, 588, 3743, 293, 309, 5541, 365, 385, 337, 257, 588, 938, 565, 293, 13038, 286, 362, 264, 2650, 51312], 'temperature': 0.0, 'avg_logprob': -0.268391489982605, 'compression_ratio': 1.5337078651685394, 'no_speech_prob': 0.003574612084776163}, {'id': 80,'seek': 50464,'start': 523.6, 'end': 533.36, 'text':'I speak in front of children and especially children in our community that they often second', 'tokens': [51312, 286, 1710, 294, 1868, 295, 2227, 293, 2318, 2227, 294, 527, 1768, 300, 436, 2049, 1150, 51800], 'temperature': 0.0, 'avg_logprob': -0.268391489982605, 'compression_ratio': 1.5337078651685394, 'no_speech_prob': 0.003574612084776163}, {'id': 81,'seek': 53336,'start': 533.36, 'end': 539.6800000000001, 'text': " guests themselves and where they're coming from, who they are and I always say that it's very", 'tokens': [50364, 9804, 2969, 293, 689, 436, 434, 1348, 490, 11, 567, 436, 366, 293, 286, 1009, 584, 300, 309, 311, 588, 50680], 'temperature': 0.0, 'avg_logprob': -0.16686672168773609, 'compression_ratio': 1.7031963470319635, 'no_speech_prob': 0.0036576450802385807}, {'id': 82,'seek': 53336,'start': 539.6800000000001, 'end': 546.16, 'text': " important to be proud of who you're from and the quote that I use Sisso Chavez is pride in one's", 'tokens': [50680, 1021, 281, 312, 4570, 295, 567, 291, 434, 490, 293, 264, 6513, 300, 286, 764, 318, 891, 78, 761, 25975, 307, 10936, 294, 472, 311, 51004], 'temperature': 0.0, 'avg_logprob': -0.16686672168773609, 'compression_ratio': 1.7031963470319635, 'no_speech_prob': 0.0036576450802385807}, {'id': 83,'seek': 53336,'start': 546.16, 'end': 553.52, 'text': " own culture does not require contempt or disrespect of another and that's very important.", 'tokens': [51004, 1065, 3713, 775, 406, 3651, 47202, 420, 27058, 295, 1071, 293, 300, 311, 588, 1021, 13, 51372], 'temperature': 0.0, 'avg_logprob': -0.16686672168773609, 'compression_ratio': 1.7031963470319635, 'no_speech_prob': 0.0036576450802385807}, {'id': 84,'seek': 53336,'start': 553.52, 'end': 557.9200000000001, 'text': " It's very important for us to recognize that no matter who we are where we come from in this", 'tokens': [51372, 467, 311, 588, 1021, 337, 505, 281, 5521, 300, 572, 1871, 567, 321, 366, 689, 321, 808, 490, 294, 341, 51592], 'temperature': 0.0, 'avg_logprob': -0.16686672168773609, 'compression_ratio': 1.7031963470319635, 'no_speech_prob': 0.0036576450802385807}, {'id': 85,'seek': 55792,'start': 557.92, 'end': 565.28, 'text': " society that your pride in your own culture doesn't require the contempt or disrespect of another.", 'tokens': [50364, 4086, 300, 428, 10936, 294, 428, 1065, 3713, 1177, 380, 3651, 264, 47202, 420, 27058, 295, 1071, 13, 50732], 'temperature': 0.0, 'avg_logprob': -0.2269989013671875, 'compression_ratio': 1.7136563876651982, 'no_speech_prob': 0.007871623151004314}, {'id': 86,'seek': 55792,'start': 565.28, 'end': 571.52, 'text':'Amen, what a year to be for that to just sit on our shoulders for a while for us to think about.', 'tokens': [50732, 14092, 11, 437, 257, 1064, 281, 312, 337, 300, 281, 445, 1394, 322, 527, 10245, 337, 257, 1339, 337, 505, 281, 519, 466, 13, 51044], 'temperature': 0.0, 'avg_logprob': -0.2269989013671875, 'compression_ratio': 1.7136563876651982, 'no_speech_prob': 0.007871623151004314}, {'id': 87,'seek': 55792,'start': 573.12, 'end': 580.88, 'text': " And so I wanted to just to thank you all, I think the commission, there's going to be a couple", 'tokens': [51124, 400, 370, 286, 1415, 281, 445, 281, 1309, 291, 439, 11, 286, 519, 264, 9221, 11, 456, 311, 516, 281, 312, 257, 1916, 51512], 'temperature': 0.0, 'avg_logprob': -0.2269989013671875, 'compression_ratio': 1.7136563876651982, 'no_speech_prob': 0.007871623151004314}, {'id': 88,'seek': 55792,'start': 580.88, 'end': 585.1999999999999, 'text':'individuals that are going to come speak thank you for your art, your lovely artwork for us to see', 'tokens': [51512, 5346, 300, 366, 516, 281, 808, 1710, 1309, 291, 337, 428, 1523, 11, 428, 7496, 15829, 337, 505, 281, 536, 51728], 'temperature': 0.0, 'avg_logprob': -0.2269989013671875, 'compression_ratio': 1.7136563876651982, 'no_speech_prob': 0.007871623151004314}, {'id': 89,'seek': 58520,'start': 586.08, 'end': 590.96, 'text': " what's in your heart and what now has become a probably going to be a very important symbol", 'tokens': [50408, 437, 311, 294, 428, 1917, 293, 437, 586, 575, 1813, 257, 1391, 516, 281, 312, 257, 588, 1021, 5986, 50652], 'temperature': 0.0, 'avg_logprob': -0.13470715568179176, 'compression_ratio': 1.7330316742081449, 'no_speech_prob': 0.011605558916926384}, {'id': 90,'seek': 58520,'start': 591.6800000000001, 'end': 597.9200000000001, 'text':'for the community and also just for the work, the daily work every single day we still have a lot', 'tokens': [50688, 337, 264, 1768, 293, 611, 445, 337, 264, 589, 11, 264, 5212, 589, 633, 2167, 786, 321, 920, 362, 257, 688, 51000], 'temperature': 0.0, 'avg_logprob': -0.13470715568179176, 'compression_ratio': 1.7330316742081449, 'no_speech_prob': 0.011605558916926384}, {'id': 91,'seek': 58520,'start': 597.9200000000001, 'end': 603.6800000000001, 'text':'of brothers and sisters whose ancestors once lived in these lands freely now stand on street', 'tokens': [51000, 295, 8452, 293, 11589, 6104, 18069, 1564, 5152, 294, 613, 5949, 16433, 586, 1463, 322, 4838, 51288], 'temperature': 0.0, 'avg_logprob': -0.13470715568179176, 'compression_ratio': 1.7330316742081449, 'no_speech_prob': 0.011605558916926384}, {'id': 92,'seek': 58520,'start': 603.6800000000001, 'end': 613.6800000000001, 'text':'corners right in poverty without access to services right without access to sobriety or even housing', 'tokens': [51288, 12413, 558, 294, 10958, 1553, 2105, 281, 3328, 558, 1553, 2105, 281, 18253, 470, 2210, 420, 754, 6849, 51788], 'temperature': 0.0, 'avg_logprob': -0.13470715568179176, 'compression_ratio': 1.7330316742081449, 'no_speech_prob': 0.011605558916926384}, {'id': 93,'seek': 61368,'start': 613.68, 'end': 622.4, 'text':'or jobs and what a what a what a cruel way to pay back a culture that has paved the way for the', 'tokens': [50364, 420, 4782, 293, 437, 257, 437, 257, 437, 257, 16022, 636, 281, 1689, 646, 257, 3713, 300, 575, 42989, 264, 636, 337, 264, 50800], 'temperature': 0.0, 'avg_logprob': -0.12040589253107707, 'compression_ratio': 1.7235023041474655, 'no_speech_prob': 0.0033086128532886505}, {'id': 94,'seek': 61368,'start': 622.4, 'end': 629.1999999999999, 'text':'city to be built upon its shores right so we have a lot of work to do and these kind of', 'tokens': [50800, 2307, 281, 312, 3094, 3564, 1080, 44247, 558, 370, 321, 362, 257, 688, 295, 589, 281, 360, 293, 613, 733, 295, 51140], 'temperature': 0.0, 'avg_logprob': -0.12040589253107707, 'compression_ratio': 1.7235023041474655, 'no_speech_prob': 0.0033086128532886505}, {'id': 95,'seek': 61368,'start': 629.1999999999999, 'end': 635.8399999999999, 'text': " proclamations in this day is not a day off it's a day on and then right and addressing those", 'tokens': [51140, 447, 3474, 335, 763, 294, 341, 786, 307, 406, 257, 786, 766, 309, 311, 257, 786, 322, 293, 550, 558, 293, 14329, 729, 51472], 'temperature': 0.0, 'avg_logprob': -0.12040589253107707, 'compression_ratio': 1.7235023041474655, 'no_speech_prob': 0.0033086128532886505}, {'id': 96,'seek': 61368,'start': 635.8399999999999, 'end': 642.64, 'text': " those those critical issues so I know that my colleagues are very supportive I'm going to ask you", 'tokens': [51472, 729, 729, 4924, 2663, 370, 286, 458, 300, 452, 7734, 366, 588, 14435, 286, 478, 516, 281, 1029, 291, 51812], 'temperature': 0.0, 'avg_logprob': -0.12040589253107707, 'compression_ratio': 1.7235023041474655, 'no_speech_prob': 0.0033086128532886505}, {'id': 97,'seek': 64264,'start': 642.64, 'end': 647.28, 'text': " to support this proclamation as I know you always have done in the past I'm very proud of today", 'tokens': [50364, 281, 1406, 341, 447, 41411, 382, 286, 458, 291, 1009, 362, 1096, 294, 264, 1791, 286, 478, 588, 4570, 295, 965, 50596], 'temperature': 0.0, 'avg_logprob': -0.1914598853499801, 'compression_ratio': 1.641304347826087, 'no_speech_prob': 0.017412053421139717}, {'id': 98,'seek': 64264,'start': 648.16, 'end': 653.76, 'text':'oh and we made time magazine and newsweek once again today as being a leader in terms of the', 'tokens': [50640, 1954, 293, 321, 1027, 565, 11332, 293, 2583, 23188, 1564, 797, 965, 382, 885, 257, 5263, 294, 2115, 295, 264, 50920], 'temperature': 0.0, 'avg_logprob': -0.1914598853499801, 'compression_ratio': 1.641304347826087, 'no_speech_prob': 0.017412053421139717}, {'id': 99,'seek': 64264,'start': 653.76, 'end': 658.4, 'text':'cities that are celebrating Indigenous peoples day I wanted to make a point out of that', 'tokens': [50920, 6486, 300, 366, 15252, 22699, 16915, 786, 286, 1415, 281, 652, 257, 935, 484, 295, 300, 51152], 'temperature': 0.0, 'avg_logprob': -0.1914598853499801, 'compression_ratio': 1.641304347826087, 'no_speech_prob': 0.017412053421139717}, {'id': 100,'seek': 64264,'start': 659.84, 'end': 664.16, 'text':'thank you uh councilman Lopez and thank you for sponsoring this council martega', 'tokens': [51224, 1309, 291, 2232, 9209, 1601, 36077, 293, 1309, 291, 337, 30311, 341, 9209, 12396, 6335, 51440], 'temperature': 0.0, 'avg_logprob': -0.1914598853499801, 'compression_ratio': 1.641304347826087, 'no_speech_prob': 0.017412053421139717}, {'id': 101,'seek': 64264,'start': 664.72, 'end': 670.16, 'text': " Mr. President I want to ask that my name be added I don't think I could um add much more to what", 'tokens': [51468, 2221, 13, 3117, 286, 528, 281, 1029, 300, 452, 1315, 312, 3869, 286, 500, 380, 519, 286, 727, 1105, 909, 709, 544, 281, 437, 51740], 'temperature': 0.0, 'avg_logprob': -0.1914598853499801, 'compression_ratio': 1.641304347826087, 'no_speech_prob': 0.017412053421139717}, {'id': 102,'seek': 67016,'start': 670.24, 'end': 677.8399999999999, 'text':'councilman Lopez has shared with us I want to thank him for bringing this forward and really', 'tokens': [50368, 9209, 1601, 36077, 575, 5507, 365, 505, 286, 528, 281, 1309, 796, 337, 5062, 341, 2128, 293, 534, 50748], 'temperature': 0.0, 'avg_logprob': -0.12131015975753982, 'compression_ratio': 1.5932203389830508, 'no_speech_prob': 0.006188048515468836}, {'id': 103,'seek': 67016,'start': 677.8399999999999, 'end': 685.6, 'text':'just appreciate all the contributions that our Native American community has contributed to this', 'tokens': [50748, 445, 4449, 439, 264, 15725, 300, 527, 15093, 2665, 1768, 575, 18434, 281, 341, 51136], 'temperature': 0.0, 'avg_logprob': -0.12131015975753982, 'compression_ratio': 1.5932203389830508, 'no_speech_prob': 0.006188048515468836}, {'id': 104,'seek': 67016,'start': 685.6, 'end': 691.28, 'text': " great city and great state I worked in the lieutenant governor's office when the commission", 'tokens': [51136, 869, 2307, 293, 869, 1785, 286, 2732, 294, 264, 45521, 12965, 311, 3398, 562, 264, 9221, 51420], 'temperature': 0.0, 'avg_logprob': -0.12131015975753982, 'compression_ratio': 1.5932203389830508, 'no_speech_prob': 0.006188048515468836}, {'id': 105,'seek': 67016,'start': 691.28, 'end': 699.28, 'text':'on Indian affairs was created and had the benefit of being able to go down to the four corners', 'tokens': [51420, 322, 6427, 17478, 390, 2942, 293, 632, 264, 5121, 295, 885, 1075, 281, 352, 760, 281, 264, 1451, 12413, 51820], 'temperature': 0.0, 'avg_logprob': -0.12131015975753982, 'compression_ratio': 1.5932203389830508, 'no_speech_prob': 0.006188048515468836}, {'id': 106,'seek': 69928,'start': 699.8399999999999, 'end': 705.4399999999999, 'text':'for a peace treaty signing ceremony between the uts and the command cheese that had been sort of at', 'tokens': [50392, 337, 257, 4336, 24772, 13393, 12813, 1296, 264, 344, 1373, 293, 264, 5622, 5399, 300, 632, 668, 1333, 295, 412, 50672], 'temperature': 0.0, 'avg_logprob': -0.144526329907504, 'compression_ratio': 1.7180616740088106, 'no_speech_prob': 0.005698710680007935}, {'id': 107,'seek': 69928,'start': 705.4399999999999, 'end': 710.4, 'text':'odds with each other for about a hundred years and just being able to participate in that powwow', 'tokens': [50672, 17439, 365, 1184, 661, 337, 466, 257, 3262, 924, 293, 445, 885, 1075, 281, 8197, 294, 300, 3388, 34354, 50920], 'temperature': 0.0, 'avg_logprob': -0.144526329907504, 'compression_ratio': 1.7180616740088106, 'no_speech_prob': 0.005698710680007935}, {'id': 108,'seek': 69928,'start': 710.4, 'end': 718.24, 'text':'was was pretty awesome so um and for those of you who continue to participate in the annual powwow', 'tokens': [50920, 390, 390, 1238, 3476, 370, 1105, 293, 337, 729, 295, 291, 567, 2354, 281, 8197, 294, 264, 9784, 3388, 34354, 51312], 'temperature': 0.0, 'avg_logprob': -0.144526329907504, 'compression_ratio': 1.7180616740088106, 'no_speech_prob': 0.005698710680007935}, {'id': 109,'seek': 69928,'start': 718.24, 'end': 725.8399999999999, 'text': " it's it's such a great opportunity for everybody else to enjoy so many of the contributions of", 'tokens': [51312, 309, 311, 309, 311, 1270, 257, 869, 2650, 337, 2201, 1646, 281, 2103, 370, 867, 295, 264, 15725, 295, 51692], 'temperature': 0.0, 'avg_logprob': -0.144526329907504, 'compression_ratio': 1.7180616740088106, 'no_speech_prob': 0.005698710680007935}, {'id': 110,'seek': 72584,'start': 725.84, 'end': 732.5600000000001, 'text':'the culture I mean to see that the dance continues to be carried on as well as as the native language', 'tokens': [50364, 264, 3713, 286, 914, 281, 536, 300, 264, 4489, 6515, 281, 312, 9094, 322, 382, 731, 382, 382, 264, 8470, 2856, 50700], 'temperature': 0.0, 'avg_logprob': -0.10793181727914249, 'compression_ratio': 1.7094972067039107, 'no_speech_prob': 0.0006760209798812866}, {'id': 111,'seek': 72584,'start': 732.5600000000001, 'end': 739.6, 'text':'from generation to generation is just so incredible because in so many cultures you know people have come', 'tokens': [50700, 490, 5125, 281, 5125, 307, 445, 370, 4651, 570, 294, 370, 867, 12951, 291, 458, 561, 362, 808, 51052], 'temperature': 0.0, 'avg_logprob': -0.10793181727914249, 'compression_ratio': 1.7094972067039107, 'no_speech_prob': 0.0006760209798812866}, {'id': 112,'seek': 72584,'start': 739.6, 'end': 749.6, 'text':'here and um assimilated to the you know the norms here and they lose their language and and lose a', 'tokens': [51052, 510, 293, 1105, 8249, 45678, 281, 264, 291, 458, 264, 24357, 510, 293, 436, 3624, 641, 2856, 293, 293, 3624, 257, 51552], 'temperature': 0.0, 'avg_logprob': -0.10793181727914249, 'compression_ratio': 1.7094972067039107, 'no_speech_prob': 0.0006760209798812866}, {'id': 113,'seek': 74960,'start': 749.6, 'end': 756.5600000000001, 'text': " lot of the culture and in the native community that that hasn't happened that has that you know", 'tokens': [50364, 688, 295, 264, 3713, 293, 294, 264, 8470, 1768, 300, 300, 6132, 380, 2011, 300, 575, 300, 291, 458, 50712], 'temperature': 0.0, 'avg_logprob': -0.15890017520175892, 'compression_ratio': 1.7962085308056872, 'no_speech_prob': 0.0026297522708773613}, {'id': 114,'seek': 74960,'start': 759.28, 'end': 766.8000000000001, 'text': " commitment to just passing that on from generation to generation is is so important and so I'm", 'tokens': [50848, 8371, 281, 445, 8437, 300, 322, 490, 5125, 281, 5125, 307, 307, 370, 1021, 293, 370, 286, 478, 51224], 'temperature': 0.0, 'avg_logprob': -0.15890017520175892, 'compression_ratio': 1.7962085308056872, 'no_speech_prob': 0.0026297522708773613}, {'id': 115,'seek': 74960,'start': 766.8000000000001, 'end': 773.0400000000001, 'text':'happy to be a co-sponsor of this tonight thank you all right thank you councilman or take a', 'tokens': [51224, 2055, 281, 312, 257, 598, 12, 4952, 892, 284, 295, 341, 4440, 1309, 291, 439, 558, 1309, 291, 9209, 1601, 420, 747, 257, 51536], 'temperature': 0.0, 'avg_logprob': -0.15890017520175892, 'compression_ratio': 1.7962085308056872, 'no_speech_prob': 0.0026297522708773613}, {'id': 116,'seek': 74960,'start': 773.0400000000001, 'end': 777.84, 'text':'councilwoman can you thank you very much and I also want to thank my colleague for bringing this', 'tokens': [51536, 9209, 18339, 393, 291, 1309, 291, 588, 709, 293, 286, 611, 528, 281, 1309, 452, 13532, 337, 5062, 341, 51776], 'temperature': 0.0, 'avg_logprob': -0.15890017520175892, 'compression_ratio': 1.7962085308056872, 'no_speech_prob': 0.0026297522708773613}, {'id': 117,'seek': 77784,'start': 777.84, 'end': 783.36, 'text':'forward and I I just wanted to to say a word to the artist about how beautiful and moving I', 'tokens': [50364, 2128, 293, 286, 286, 445, 1415, 281, 281, 584, 257, 1349, 281, 264, 5748, 466, 577, 2238, 293, 2684, 286, 50640], 'temperature': 0.0, 'avg_logprob': -0.0758090615272522, 'compression_ratio': 1.7984790874524714, 'no_speech_prob': 0.001090412144549191}, {'id': 118,'seek': 77784,'start': 783.36, 'end': 789.6, 'text':'thought this logo was and your description of it um and I think one of the things um that is', 'tokens': [50640, 1194, 341, 9699, 390, 293, 428, 3855, 295, 309, 1105, 293, 286, 519, 472, 295, 264, 721, 1105, 300, 307, 50952], 'temperature': 0.0, 'avg_logprob': -0.0758090615272522, 'compression_ratio': 1.7984790874524714, 'no_speech_prob': 0.001090412144549191}, {'id': 119,'seek': 77784,'start': 789.6, 'end': 794.72, 'text': " clear is you know the words sometimes don't convey the power of imagery or music or the other", 'tokens': [50952, 1850, 307, 291, 458, 264, 2283, 2171, 500, 380, 16965, 264, 1347, 295, 24340, 420, 1318, 420, 264, 661, 51208], 'temperature': 0.0, 'avg_logprob': -0.0758090615272522, 'compression_ratio': 1.7984790874524714, 'no_speech_prob': 0.001090412144549191}, {'id': 120,'seek': 77784,'start': 794.72, 'end': 800.72, 'text':'pieces that make up culture and so I think the art is so important and when you talked about water', 'tokens': [51208, 3755, 300, 652, 493, 3713, 293, 370, 286, 519, 264, 1523, 307, 370, 1021, 293, 562, 291, 2825, 466, 1281, 51508], 'temperature': 0.0, 'avg_logprob': -0.0758090615272522, 'compression_ratio': 1.7984790874524714, 'no_speech_prob': 0.001090412144549191}, {'id': 121,'seek': 77784,'start': 800.72, 'end': 805.84, 'text':'I was also thinking about land and I guess I just wanted to say thank you um many of the native', 'tokens': [51508, 286, 390, 611, 1953, 466, 2117, 293, 286, 2041, 286, 445, 1415, 281, 584, 1309, 291, 1105, 867, 295, 264, 8470, 51764], 'temperature': 0.0, 'avg_logprob': -0.0758090615272522, 'compression_ratio': 1.7984790874524714, 'no_speech_prob': 0.001090412144549191}, {'id': 122,'seek': 80584,'start': 805.84, 'end': 810.48, 'text':'American peoples of Colorado have been at the forefront are actually nationally of defending some', 'tokens': [50364, 2665, 16915, 295, 15786, 362, 668, 412, 264, 27287, 366, 767, 27652, 295, 21377, 512, 50596], 'temperature': 0.0, 'avg_logprob': -0.09510297873585495, 'compression_ratio': 1.7481751824817517, 'no_speech_prob': 0.002177680842578411}, {'id': 123,'seek': 80584,'start': 810.48, 'end': 815.52, 'text':'of the the um public lands that have been protected over the last few years that are under attack', 'tokens': [50596, 295, 264, 264, 1105, 1908, 5949, 300, 362, 668, 10594, 670, 264, 1036, 1326, 924, 300, 366, 833, 2690, 50848], 'temperature': 0.0, 'avg_logprob': -0.09510297873585495, 'compression_ratio': 1.7481751824817517, 'no_speech_prob': 0.002177680842578411}, {'id': 124,'seek': 80584,'start': 815.52, 'end': 821.76, 'text':'right now and their places that you uh the communities have fought to protect but that everyone', 'tokens': [50848, 558, 586, 293, 641, 3190, 300, 291, 2232, 264, 4456, 362, 11391, 281, 2371, 457, 300, 1518, 51160], 'temperature': 0.0, 'avg_logprob': -0.09510297873585495, 'compression_ratio': 1.7481751824817517, 'no_speech_prob': 0.002177680842578411}, {'id': 125,'seek': 80584,'start': 821.76, 'end': 828.32, 'text': " gets to enjoy and so I just think that it's an example of where cultural preservation intersects", 'tokens': [51160, 2170, 281, 2103, 293, 370, 286, 445, 519, 300, 309, 311, 364, 1365, 295, 689, 6988, 27257, 27815, 82, 51488], 'temperature': 0.0, 'avg_logprob': -0.09510297873585495, 'compression_ratio': 1.7481751824817517, 'no_speech_prob': 0.002177680842578411}, {'id': 126,'seek': 80584,'start': 828.32, 'end': 833.6, 'text':'with environmental protection with you know recreation and all of the other ways that that', 'tokens': [51488, 365, 8303, 6334, 365, 291, 458, 31573, 293, 439, 295, 264, 661, 2098, 300, 300, 51752], 'temperature': 0.0, 'avg_logprob': -0.09510297873585495, 'compression_ratio': 1.7481751824817517, 'no_speech_prob': 0.002177680842578411}, {'id': 127,'seek': 83360,'start': 833.6800000000001, 'end': 838.32, 'text':'public lands are so important and so I think um I just wanted to say thank you for that because I', 'tokens': [50368, 1908, 5949, 366, 370, 1021, 293, 370, 286, 519, 1105, 286, 445, 1415, 281, 584, 1309, 291, 337, 300, 570, 286, 50600], 'temperature': 0.0, 'avg_logprob': -0.09590956870089756, 'compression_ratio': 1.7935779816513762, 'no_speech_prob': 0.0007911771535873413}, {'id': 128,'seek': 83360,'start': 838.32, 'end': 844.5600000000001, 'text':'think we have some very sacred places in our country that are at risk right now and um so as we', 'tokens': [50600, 519, 321, 362, 512, 588, 15757, 3190, 294, 527, 1941, 300, 366, 412, 3148, 558, 586, 293, 1105, 370, 382, 321, 50912], 'temperature': 0.0, 'avg_logprob': -0.09590956870089756, 'compression_ratio': 1.7935779816513762, 'no_speech_prob': 0.0007911771535873413}, {'id': 129,'seek': 83360,'start': 844.5600000000001, 'end': 849.44, 'text': " celebrate I appreciate that there's still a piece of resistance in here and I think that um I", 'tokens': [50912, 8098, 286, 4449, 300, 456, 311, 920, 257, 2522, 295, 7335, 294, 510, 293, 286, 519, 300, 1105, 286, 51156], 'temperature': 0.0, 'avg_logprob': -0.09590956870089756, 'compression_ratio': 1.7935779816513762, 'no_speech_prob': 0.0007911771535873413}, {'id': 130,'seek': 83360,'start': 849.44, 'end': 855.0400000000001, 'text':'just want to mention a solidarity and I mentioned a feeling of solidarity with that resistance so thank', 'tokens': [51156, 445, 528, 281, 2152, 257, 27220, 293, 286, 2835, 257, 2633, 295, 27220, 365, 300, 7335, 370, 1309, 51436], 'temperature': 0.0, 'avg_logprob': -0.09590956870089756, 'compression_ratio': 1.7935779816513762, 'no_speech_prob': 0.0007911771535873413}, {'id': 131,'seek': 85504,'start': 855.04, 'end': 863.52, 'text':'you and uh happy confluence week thank you uh councilman can each uh and see you know other', 'tokens': [50364, 291, 293, 2232, 2055, 1497, 40432, 1243, 1309, 291, 2232, 9209, 1601, 393, 1184, 2232, 293, 536, 291, 458, 661, 50788], 'temperature': 0.0, 'avg_logprob': -0.20807060073403752, 'compression_ratio': 1.532258064516129, 'no_speech_prob': 0.0049024922773242}, {'id': 132,'seek': 85504,'start': 863.52, 'end': 873.5999999999999, 'text': " comments I'll just say a couple and um in a time of such divisive um ugliness and just despicable", 'tokens': [50788, 3053, 286, 603, 445, 584, 257, 1916, 293, 1105, 294, 257, 565, 295, 1270, 25974, 488, 1105, 344, 7191, 1324, 293, 445, 4887, 43023, 51292], 'temperature': 0.0, 'avg_logprob': -0.20807060073403752, 'compression_ratio': 1.532258064516129, 'no_speech_prob': 0.0049024922773242}, {'id': 133,'seek': 85504,'start': 873.5999999999999, 'end': 880.0799999999999, 'text': " behavior from our leadership um the reason I'm so supportive of indigenous peoples days because", 'tokens': [51292, 5223, 490, 527, 5848, 1105, 264, 1778, 286, 478, 370, 14435, 295, 15511, 16915, 1708, 570, 51616], 'temperature': 0.0, 'avg_logprob': -0.20807060073403752, 'compression_ratio': 1.532258064516129, 'no_speech_prob': 0.0049024922773242}, {'id': 134,'seek': 88008,'start': 880.08, 'end': 885.84, 'text':'it means inclusivity it means respecting all respecting those who have been silenced', 'tokens': [50364, 309, 1355, 17204, 4253, 309, 1355, 41968, 439, 41968, 729, 567, 362, 668, 3425, 14672, 50652], 'temperature': 0.0, 'avg_logprob': -0.15961699258713496, 'compression_ratio': 1.5677966101694916, 'no_speech_prob': 0.0006969001842662692}, {'id': 135,'seek': 88008,'start': 886.5600000000001, 'end': 894.48, 'text':'um uh on purpose for a long time and whose history has not been told and so we celebrate inclusivity', 'tokens': [50688, 1105, 2232, 322, 4334, 337, 257, 938, 565, 293, 6104, 2503, 575, 406, 668, 1907, 293, 370, 321, 8098, 17204, 4253, 51084], 'temperature': 0.0, 'avg_logprob': -0.15961699258713496, 'compression_ratio': 1.5677966101694916, 'no_speech_prob': 0.0006969001842662692}, {'id': 136,'seek': 89448,'start': 894.48, 'end': 908.4, 'text':'in the face of such um such evil types honestly', 'tokens': [50364, 294, 264, 1851, 295, 1270, 1105, 1270, 6724, 3467, 6095, 51060], 'temperature': 0.0, 'avg_logprob': -0.48870185705331654, 'compression_ratio': 0.94, 'no_speech_prob': 0.11038731038570404}], 'language': 'en'}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

**Minutes of the Denver City Council Meeting**

**Date:** October 9, 2017
**Location:** Denver City Council Chambers
**Attendees:**

* Councilman Lopez
* Councilman Clark
* Councilwoman Ortega
* Councilman Cashman
* Councilman Can Each
* Councilman Sussan
* Councilman Flynn
* Councilman Gilmore
* Councilman Espinosa
* Councilman Black

**Summary:**

The Denver City Council met on October 9, 2017, to discuss various matters, including the approval of the minutes from the previous meeting, announcements, presentations, communications, and proclamations. The council also celebrated Indigenous Peoples Day and recognized the contributions of Indigenous people to the city's history and culture.

**Key Discussion Points:**

* The council approved the minutes from the previous meeting without any corrections.
* Councilman Clark announced the first-ever Halloween parade in the Lucky District 7, which will take place on October 21, 2017.
* The council adopted Proclamation 1127, Series of 2017, which observes the second annual Indigenous Peoples Day in the City and County of Denver.
* Councilman Lopez read the proclamation, highlighting the importance of recognizing and valuing the contributions of Indigenous people to the community.
* Councilwoman Ortega and Councilman Can Each spoke in support of the proclamation, emphasizing the need to promote education and awareness about Indigenous peoples' history and culture.
* Councilman Cashman expressed his solidarity with the Indigenous community and highlighted the importance of preserving cultural heritage and protecting public lands.
* Councilman Sussan mentioned the need to address critical issues affecting Indigenous communities, such as poverty, lack of access to services, and housing.

**Takeaways:**

* The council recognized the importance of Indigenous Peoples Day and the need to promote education and awareness about Indigenous peoples' history and culture.
* The council acknowledged the contributions of Indigenous people to the city's history and culture.
* The council emphasized the need to address critical issues affecting Indigenous communities.

**Action Items:**

* Councilman Lopez will work with the council to implement the recommendations from the proclamation.
* Councilwoman Ortega and Councilman Can Each will collaborate with the council to promote education and awareness about Indigenous peoples' history and culture.
* Councilman Cashman will work with the council to address critical issues affecting Indigenous communities.

**Next Steps:**

* The council will continue to work on implementing the recommendations from the proclamation.
* The council will schedule a follow-up meeting to discuss the progress made on addressing critical issues affecting Indigenous communities.<|eot_id|>

# Student contribution

Student Emad S. has made this powerful variation that uses `TextIteratorStreamer` to stream back results into a Gradio UI, and takes advantage of background threads for performance! I'm sharing it here if you'd like to take a look at some very interesting work. Thank you, Emad!

https://colab.research.google.com/drive/1Ja5zyniyJo5y8s1LKeCTSkB2xyDPOt6D

## Alternative implementation

Class student Youssef has contributed this variation in which we use an open-source model to transcribe the meeting Audio.

Thank you Youssef!

In [16]:
from transformers import AutoModelForSpeechSeq2Seq


In [17]:
AUDIO_MODEL = "openai/whisper-medium"
speech_model = AutoModelForSpeechSeq2Seq.from_pretrained(AUDIO_MODEL, torch_dtype=torch.float16, low_cpu_mem_usage=True, use_safetensors=True)
speech_model.to('cuda')
processor = AutoProcessor.from_pretrained(AUDIO_MODEL)

pipe = pipeline(
    "automatic-speech-recognition",
    model=speech_model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device='cuda',
)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

NameError: name 'AutoProcessor' is not defined

In [ ]:
# Use the Whisper OpenAI model to convert the Audio to Text
result = pipe(audio_filename)

In [ ]:
transcription = result["text"]
print(transcription)